Please remember to update the token. For multiplayer to work, each player needs a different token. Please feel free to use multiple tokens in the same group.

In [1]:
#Update your token
STUDENT_TOKEN = 'Juan Rubiales'

There should be no need to change the code below.

In [2]:
## ignore this code, just used for submission
import requests
import pprint
import json
import random
import time
from copy import copy, deepcopy

class Game:
  def __init__(self, state, status, player):
    self.state = state
    self.status = status
    self.player = player

  def is_waiting(self):
    return self.status == 'waiting'

  def is_end(self):
    return self.status == 'complete'
  
  def get_board(self):
    return json.loads(self.state)

  def get_winner(self):
    return None

  def actions(self):
    return []

  def print(self):
    print(self.state)

def new_game(game_type, multi_player = False):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/new-game?TOKEN=%s&game-type=%s&multi-player=%s' % (STUDENT_TOKEN, game_type, 'True' if multi_player else 'False'))
    if r.status_code == 200:
      return r.json()['game-id']
    print(r.content)

def join_game(game_type, game_id):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/join-game?TOKEN=%s&game-type=%s&game-id=%s' % (STUDENT_TOKEN, game_type, game_id))
    if r.status_code == 200:
      return r.json()['player']
    print(r.content)

def game_state(game_type, game_id, GameClass):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/game-state?TOKEN=%s&game-type=%s&game-id=%s' % (STUDENT_TOKEN, game_type, game_id))
    if r.status_code == 200:
      return GameClass(r.json()['state'], r.json()['status'], r.json()['player'])
    print(r.content)

def update_game(game_type, game_id, player, move):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/update-game?TOKEN=%s&game-type=%s&game-id=%s&player=%s&move=%s' % (STUDENT_TOKEN, game_type, game_id, player, move))
    if r.status_code == 200:
      return r.content
    print(r.content)

def game_loop(solver, GameClass, game_type, multi_player = False, id = None):
  while id == None:
    print('Creating new game...')
    id = new_game(game_type, multi_player)

  print('Joining game with id: %s' % id)
  player = join_game(game_type, id)

  print('Playing as %s' % player)

  game = game_state(game_type, id, GameClass)
  print('Waiting for the other player to join...')
  while game.is_waiting():
    time.sleep(10)
    game = game_state(game_type, id, GameClass)

  while True:
    game = game_state(game_type, id, GameClass)
    game.print_game()
    if game.is_end():
      if game.player == '-':
        print('draw')
      else:
        print('winner' if game.player == player else 'loser')
      return
    if game.player == player:
      print('Making next move...')
      move = solver(game)
      update_result = update_game(game_type, id, player, json.dumps(move))
      print(update_result)
    else:
      time.sleep(2)

There is almost certainly no need to change this code, but if you must you can.

In [3]:
from functools import reduce
from copy import copy, deepcopy
import json
import random

class ConnectFour(Game):
  def __init__(self, state, status, player):
    Game.__init__(self, state, status, player)

  def actions(self):
    return [] # this should return the possible actions

  def get_winner(self):
    return '.' # this should return the actual winner

  def other_player(self):
    if self.player == 'O': return 'X'
    if self.player == 'X': return 'O'

  def print_game(self):

    for row in self.get_board():
      print(row)
    # print(self.state)

Instead of changing this code for a random solver, write new code that also returns the next move. For example, create a new "amazing_solver"

In [4]:
def random_solver(game):
  board = game.get_board()
  while True:
      x = 0
      y = random.randint(0, len(board[0]) - 1)
      if board[x][y] == '.':
        print('x: %s, y: %s' % (x, y))
        return y
  return None

In [5]:
# Heuristic for the Connect Four game
# Maximize the number of consecutive pieces in a row
# Minimize the number of consecutive pieces in a row for the opponent
# Maximize the number of consecutive pieces in a column
# Minimize the number of consecutive pieces in a column for the opponent
# Maximize the number of consecutive pieces in a diagonal
# Minimize the number of consecutive pieces in a diagonal for the opponent


# j is the index of the last piece played
def heuristic(game, j):
    board = game.get_board()
    player = game.player
    opponent = game.other_player()
    score = 0
    
    # Check the row
    row = board[0][j]
    score += heuristic_row(row, player)
    score -= heuristic_row(row, opponent)
    #  If the player has a move that creates a 3 in a row
    #  then it is a winning move
    count = 0
    for i in range(1, len(board)):
        if board[i][j] == player:
            count += 1
    if count == 3:
        return 1000

    # Check if the opponent has a move that creates a 3 in a row
    # If so, then it is a losing move
    count = 0
    for i in range(1, len(board)):
        if board[i][j] == opponent:
            count += 1
    if count == 3:
        return 1000


    
    # If the column is full, return the score
    if board[0][j] != '.':
        return 0
    # Check the column
    column = [row[j] for row in board]
    score += heuristic_row(column, player)
    score -= heuristic_row(column, opponent)

    # If the player has a move that creates a 3 in a column
    # then it is a winning move
    count = 0
    for i in range(1, len(board)):
        
        if board[i][j] == player:
            count += 1
    if count == 3:
        return 1000

    # Check if the opponent has a move that creates a 3 in a column
    # If so, then it is a losing move
    count = 0
    for i in range(1, len(board)):
        if board[i][j] == opponent:
            count += 1
    if count == 3:
        return 1000

    
    # Check if j makes  a diagonal
    # If so, check the diagonal
    if j == 0 or j == 1:
        diagonal = [board[i][j+i] for i in range(len(board))]
        score += heuristic_row(diagonal, player)
        score -= heuristic_row(diagonal, opponent)

        # If the player has a move that creates a 3 in a diagonal
        # then it is a winning move
        count = 0
        for i in range(1, len(board)):
            if board[i][j+i] == player:
                count += 1
        if count == 3:
            return 1000

        # Check if the opponent has a move that creates a 3 in a diagonal
        # If so, then it is a losing move
        count = 0
        for i in range(1, len(board)):
            if board[i][j+i] == opponent:
                count += 1
        if count == 3:
            return 1000

    if j == 3 or j == 4:
        diagonal = [board[i][j-i] for i in range(len(board))]
        score += heuristic_row(diagonal, player)
        score -= heuristic_row(diagonal, opponent)

        # If the player has a move that creates a 3 in a diagonal
        # then it is a winning move
        count = 0
        for i in range(1, len(board)):
            if board[i][j-i] == player:
                count += 1
        if count == 3:
            return 1000

        # Check if the opponent has a move that creates a 3 in a diagonal
        # If so, then it is a losing move
        count = 0
        for i in range(1, len(board)):
            if board[i][j-i] == opponent:
                count += 1
        if count == 3:
            return 1000


    # Check if j makes  a negative diagonal
    # If so, check the diagonal
    if j == 3 or j == 4:
        diagonal = [board[i][j-i] for i in range(len(board))]
        score += heuristic_row(diagonal, player)
        score -= heuristic_row(diagonal, opponent)

        # If the player has a move that creates a 3 in a diagonal
        # then it is a winning move
        count = 0
        for i in range(1, len(board)):
            if board[i][j-i] == player:
                count += 1
        if count == 3:
            return 1000

        # Check if the opponent has a move that creates a 3 in a diagonal
        # If so, then it is a losing move
        count = 0
        for i in range(1, len(board)):
            if board[i][j-i] == opponent:
                count += 1
        if count == 3:
            return 1000

    if j == 0 or j == 1:
        diagonal = [board[i][j+i] for i in range(len(board))]
        score += heuristic_row(diagonal, player)
        score -= heuristic_row(diagonal, opponent)

        # If the player has a move that creates a 3 in a diagonal
        # then it is a winning move
        count = 0
        for i in range(1, len(board)):
            if board[i][j+i] == player:
                count += 1
        if count == 3:
            return 1000

        # Check if the opponent has a move that creates a 3 in a diagonal
        # If so, then it is a losing move
        count = 0
        for i in range(1, len(board)):
            if board[i][j+i] == opponent:
                count += 1
        if count == 3:
            return 1000

            


    return score

def heuristic_row(row, player):
    score = 0
    consecutive = 0
    for i in range(len(row)):
        if row[i] == player:
            consecutive += 1
        else:
            score += consecutive**2
            consecutive = 0
    score += consecutive**2
    return score
    
# function to find the best possible move for the given game state with heuristic
# 2 minutes per play,

def heuristic_solver(game):
    board = game.get_board()
    best_score = -1000000
    best_move = None

    score_table = []
    for i in range(len(board[0])):
        print("position", i)
        if board[0][i] == '.':
            board[0][i] = game.player
            score = heuristic(game, i)
            print('score: %s' % score)
            score_table.append((score, i))
            if score > best_score:
                # print('best score: %s' % score)
                best_score = score
                best_move = i
            board[0][i] = '.'
    print(score_table)
    return best_move
    
            

This code is the one that starts the game. In different games you will need to change the class (ConnectFour is the one for connect four game). In connect four, there are two game types, 'connect4' for a small board, and 'connect4big' for a big board.

The 'multi_player' variable can be True or False. If you set multi_palyer to True, you will need to provide a game_id against which to play or if using None a new game will be created (which another player can join).

In [6]:
game_loop(heuristic_solver, ConnectFour, 'connect4big', multi_player=False)

Creating new game...
Joining game with id: 190
Playing as O
Waiting for the other player to join...
['.', '.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.', '.']
['.', '.', '.', 'X', '.', '.', '.']
Making next move...
position 0
score: 0
position 1
score: 0
position 2
score: 0
position 3
score: -1
position 4
score: 0
position 5
score: 0
position 6
score: 0
[(0, 0), (0, 1), (0, 2), (-1, 3), (0, 4), (0, 5), (0, 6)]
b'Valid move'
['.', '.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.', '.']
['O', '.', '.', 'X', '.', 'X', '.']
Making next move...
position 0
score: -1
position 1
score: 0
position 2
score: 0
position 3
score: -3
position 4
score: 0
position 5
score: -1
position 6
score: 0
[(-1, 0), (0, 1), (0, 2), (-3, 3), (0, 4), (-1, 5), (0, 6)]
b'Valid move

In [7]:
game_loop(heuristic_solver, ConnectFour, 'connect4big', multi_player=True, id=187)

Joining game with id: 187


KeyError: 'player'

In [ ]:
game_loop(random_solver, ConnectFour, 'connect4big', multi_player=False, id=None)

Creating new game...
Joining game with id: 108
Playing as X
Waiting for the other player to join...
['.', '.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.', '.']
Making next move...
x: 0, y: 4
b'Valid move'
['.', '.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', 'X', '.', 'O']
Making next move...
x: 0, y: 1
b'Valid move'
['.', '.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.', '.']
['O', 'X', '.', '.', 'X', '.', 'O']
Making next move...
x: 0, y: 1
b'Valid move'
['.', '.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.', '.']
['.', '.'